# Relational Data 

In this task, an agent is given access to a set of tools that can be used to make queries across 3 relational tables.

The tables contain information about users, locations and foods. The agent must answer questions about the data using the provided tools.

The underlying data looks like this:

User data:

```json
{
    "id": 1,
    "name": "Alice",
    "email": "alice@gmail.com",
    "location": 1,
    "favorite_color": "red",
    "favorite_foods": [1, 2, 3],
},
{
    "id": 21,
    "name": "Bob",
    "email": "bob@hotmail.com",
    "location": 2,
    "favorite_color": "orange",
    "favorite_foods": [4, 5, 6],
}
```

Food data:

```json
{
    "id": 1,
    "name": "Pizza",
    "calories": 285,  # Calories per serving
    "allergic_ingredients": ["Gluten", "Dairy"],
},
{
    "id": 2,
    "name": "Chocolate",
    "calories": 50,  # Calories per serving
    "allergic_ingredients": ["Milk", "Soy"],
},
```

The tools allow to look up information based on ids (e.g., `get_user_email` takes a user id and returns the email),
and to search (e.g., `find_foods_by_name` takes a food name and returns a list of results.

----------

In [1]:
from langchain_benchmarks import clone_public_dataset, registry

For this code to work, please configure LangSmith environment variables with your credentials.

In [2]:
task = registry["Tool Usage - Relational Data"]

Clone the dataset associaetd with this task

In [3]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Tool Usage - Relational Data already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/69c0e0d0-91b5-4183-bed0-6628e76964dc.


## The Environment

Let's check the environment

In [4]:
env = task.create_environment()
env.tools[:5]

[StructuredTool(name='get_user_name', description="get_user_name(user_id: int) -> str - Get the name of the user with the given user ID.\n\n        Args:\n            user_id: The user's ID.\n\n        Returns:\n            The user's name.", args_schema=<class 'pydantic.v1.main.get_user_nameSchemaSchema'>, func=<function get_available_functions.<locals>.get_user_name at 0x7f45ecc6e700>),
 StructuredTool(name='list_user_ids', description='list_user_ids() -> List[str] - List all the user IDs.', args_schema=<class 'pydantic.v1.main.list_user_idsSchemaSchema'>, func=<function get_available_functions.<locals>.list_user_ids at 0x7f45ecc6ee50>),
 StructuredTool(name='find_users_by_name', description='find_users_by_name(name: str) -> List[langchain_benchmarks.tool_usage.tasks.relational_data.SearchHit] - Find users with the given name.\n\n        Args:\n            name: The name to search for.\n\n        Returns:\n            The list of matching users.', args_schema=<class 'pydantic.v1.main

In [5]:
env.tools[0].invoke({"user_id": 21})

'Bob'

In [6]:
env.tools[3].invoke({"city": "LA"})

[{'id': 2, 'city': 'Los Angeles'},
 {'id': 1, 'city': 'New York'},
 {'id': 3, 'city': 'Chicago'},
 {'id': 4, 'city': 'Houston'},
 {'id': 5, 'city': 'Miami'}]

## Agent Factory

For evaluation, we need to create an agent factory that returns an `AgentExecutor` with an adapter.

The evaluation call will use the factory to create a new agent instance for every run.

In [7]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chat_models import ChatOpenAI

from langchain_benchmarks.schema import ExtractionTask
from langchain_benchmarks.tool_usage.agents import apply_agent_executor_adapter

In [8]:
class AgentFactory:
    def __init__(self, task: ExtractionTask, model: str) -> None:
        self.task = task
        self.model = model


    def __call__(self):
        # This factory creates a new environment for every agent run.
        # The reason is that the environment may be associated with an environment state (e.g., typewriter)
        # which is changed by the actions of the agent.
        # At the end of the run, the environment state will be read.
        env = task.create_environment()  # Create a new environment for every agent run!
        tools = env.tools
        llm = ChatOpenAI(temperature=0, model=self.model)
        agent_executor = initialize_agent(
            tools, llm, agent=AgentType.OPENAI_FUNCTIONS, return_intermediate_steps=True
        )
        # Apply the adapters so that inputs and outputs match dataset schema
        # state_reader automatically adds the state of the environment at the end of the run.
        return apply_agent_executor_adapter(agent_executor, state_reader=env.read_state)

In [9]:
models = ["gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-4-32k-0613"]
agent_factory = AgentFactory(task, models[0])

Let's test that our agent works

In [10]:
from langchain import globals
globals.set_verbose(True)

In [11]:
agent = agent_factory()
agent.invoke({"question": "whats the weather in LA?"})



> Entering new AgentExecutor chain...

Invoking: `find_locations_by_name` with `{'city': 'Los Angeles'}`


[{'id': 2, 'city': 'Los Angeles'}, {'id': 4, 'city': 'Houston'}, {'id': 1, 'city': 'New York'}, {'id': 3, 'city': 'Chicago'}, {'id': 5, 'city': 'Miami'}]
Invoking: `get_weather_at_location` with `{'location_id': 2}`


Sunny, Temperature: 75°FThe weather in Los Angeles is sunny with a temperature of 75°F.

> Finished chain.


{'question': 'whats the weather in LA?',
 'input': 'whats the weather in LA?',
 'output': 'The weather in Los Angeles is sunny with a temperature of 75°F.',
 'intermediate_steps': [(AgentActionMessageLog(tool='find_locations_by_name', tool_input={'city': 'Los Angeles'}, log="\nInvoking: `find_locations_by_name` with `{'city': 'Los Angeles'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"city":"Los Angeles"}', 'name': 'find_locations_by_name'}})]),
   [{'id': 2, 'city': 'Los Angeles'},
    {'id': 4, 'city': 'Houston'},
    {'id': 1, 'city': 'New York'},
    {'id': 3, 'city': 'Chicago'},
    {'id': 5, 'city': 'Miami'}]),
  (AgentActionMessageLog(tool='get_weather_at_location', tool_input={'location_id': 2}, log="\nInvoking: `get_weather_at_location` with `{'location_id': 2}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"location_id":2}', 'name': 'get_weather_at_location'}})]),
   'Sunny, T

In [12]:
globals.set_verbose(False)

## Eval

Let's evaluate an agent now

In [13]:
from langsmith.client import Client

from langchain_benchmarks.tool_usage import get_eval_config

In [14]:
client = Client()

In [ ]:
eval_config = get_eval_config()

test_runs = []

for model in ["gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-4-32k-0613"]:
    print(model)
    agent_factory = AgentFactory(task, model)

    test_run = client.run_on_dataset(
        dataset_name=task.name,
        llm_or_chain_factory=agent_factory,
        evaluation=eval_config,
        verbose=True,
        tags=[model],
    )
    test_runs.append(test_run)

gpt-3.5-turbo-1106
View the evaluation results for project 'test-flowery-tree-7' at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/projects/p/49b024d5-ea28-45f0-8eeb-802e5dfdb32c?eval=true

View all tests for Dataset Tool Usage - Relational Data at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/69c0e0d0-91b5-4183-bed0-6628e76964dc
[->                                                ] 1/21

Chain failed for example bf526f15-26a5-40be-8d0f-13d270c0d235 with inputs {'question': 'do bob and alice live in the same city?'}
Error Type: ValueError, Message: User ID 36 cannot be resolved


[----------------------------------------------->  ] 20/21

## Inspect

Here, we'll take a look at the underlying results a little bit.

In [ ]:
import pandas as pd

df = test_run.to_dataframe()
df = pd.json_normalize(df.to_dict(orient="records"))

df["correctness"].mean()

In [29]:
df["num_expected_steps"] = df["reference.expected_steps"].apply(len)
df["actual_number_of_steps"] = df["output.intermediate_steps"].apply(len)

df.head()

,Intermediate steps correctness,# steps / # expected steps,correctness,execution_time,input.question,output.question,output.output,output.intermediate_steps,reference.reference,reference.order_matters,reference.expected_steps,num_expected_steps,actual_number_of_steps
0,1,1.0,1,4.253613,do bob and alice live in the same city?,do bob and alice live in the same city?,"No, Bob and Alice do not live in the same city...",[(tool='find_users_by_name' tool_input={'name'...,no,False,"[find_users_by_name, get_user_location, get_ci...",5,5
1,0,0.0,0,4.253613,Is it likely that Donna is outside with an umb...,Is it likely that Donna is outside with an umb...,"I'm sorry, but I don't have access to real-tim...",[],yes,False,"[find_users_by_name, get_user_location, get_cu...",4,0
2,1,1.0,1,4.253613,do alice and charlie use the same email provider?,do alice and charlie use the same email provider?,"No, Alice and Charlie do not use the same emai...",[(tool='find_users_by_name' tool_input={'name'...,no,True,"[find_users_by_name, get_user_email, get_user_...",3,3
3,0,0.0,0,4.253613,Is it likely that Donna is awake right now?,Is it likely that Donna is awake right now?,"I'm sorry, but I don't have access to informat...",[],yes,True,"[find_users_by_name, get_user_location, get_cu...",3,0
4,0,1.0,1,4.253613,Donna is about to go outside. Does she need an...,Donna is about to go outside. Does she need an...,Donna is at location 4 and the current weather...,[(tool='find_users_by_name' tool_input={'name'...,yes,True,"[find_users_by_name, get_user_location, get_cu...",3,3


In [32]:
df.sort_values("actual_number_of_steps", ascending=False).head()

,Intermediate steps correctness,# steps / # expected steps,correctness,execution_time,input.question,output.question,output.output,output.intermediate_steps,reference.reference,reference.order_matters,reference.expected_steps,num_expected_steps,actual_number_of_steps
0,1,1.0,1,4.253613,do bob and alice live in the same city?,do bob and alice live in the same city?,"No, Bob and Alice do not live in the same city...",[(tool='find_users_by_name' tool_input={'name'...,no,False,"[find_users_by_name, get_user_location, get_ci...",5,5
2,1,1.0,1,4.253613,do alice and charlie use the same email provider?,do alice and charlie use the same email provider?,"No, Alice and Charlie do not use the same emai...",[(tool='find_users_by_name' tool_input={'name'...,no,True,"[find_users_by_name, get_user_email, get_user_...",3,3
4,0,1.0,1,4.253613,Donna is about to go outside. Does she need an...,Donna is about to go outside. Does she need an...,Donna is at location 4 and the current weather...,[(tool='find_users_by_name' tool_input={'name'...,yes,True,"[find_users_by_name, get_user_location, get_cu...",3,3
5,0,1.0,0,4.253613,whats the name of the city where bob lives?,whats the name of the city where bob lives?,The name of the city where Bob lives is New York.,[(tool='list_user_ids' tool_input={} log='\nIn...,Los Angeles,True,"[find_users_by_name, get_user_location, get_ci...",3,3
6,1,1.0,1,4.253613,what is the current users favorite color and n...,what is the current users favorite color and n...,The current user's favorite color is yellow an...,[(tool='get_current_user_id' tool_input={} log...,yellow and Charlie,True,"[get_current_user_id, get_user_favorite_color,...",3,3
